In [ ]:
def publish_post(post_manager: IgPostManager, row: pd.Series):
    """
    Publishes a post based on the data in the row.
    """
    post_type = row["post_type"].lower()
    media_path = row["Video/photo path"]
    caption = row["Caption"]
    hashtags = row["Hashtags"]
    mentions = row["Mentions"]
    
    print("post_type:", post_type)
    print(media_path)
    print(caption)
    print(hashtags)
    print(mentions)

    location_pk = int(row.get("Location"))   # Get location PK from the row
    location = ig_utils.get_location_by_pk(location_pk)
       
    location_dict = ig_utils.location_to_dict(location)
    print("location_pk: ", location_pk, "followed by the location.json")
    print(json.dump(location_dict, f, indent=4))
    
    if post_type == "photo":
        return post_manager.upload_photo(media_path, caption, location)
    elif post_type == "video":
        return post_manager.upload_video(media_path, caption, location_pk, hashtags=hashtags, mentions=mentions)
    elif post_type == "album":
        media_paths = media_path.split(",")
        return post_manager.upload_album(media_paths, caption, location_pk, hashtags=hashtags, mentions=mentions)
    else:
        raise ValueError(f"Invalid post type: {post_type}")


In [ ]:
def load_and_merge_post_history(new_posts_df):
    """
    Loads existing post history from a JSON file, merges it with new posts,
    and returns the combined DataFrame.
    """
    post_history_file = config.get("post_history_file")

    if os.path.exists(post_history_file):
        with open(post_history_file, "r", encoding="utf-8") as f:
            existing_posts = json.load(f).get("posts", [])
        existing_df = create_post_dataframe(
            [IgPost(**post_data) for post_data in existing_posts]
        )
        df_posts = pd.concat([existing_df, new_posts_df], ignore_index=True)
    else:
        df_posts = new_posts_df
        df_posts["published"] = False
        df_posts["failed_attempts"] = 0
        df_posts["last_failed_attempt"] = None

    return df_posts

## Main


In [1]:

"""
main.py:
Schedules and publishes Instagram posts from an Excel spreadsheet,
managing post history and retrying failed uploads.
"""

import os
import json
import random
import time
import datetime
import logging

import pandas as pd
from pathlib import Path

from instagrapi.exceptions import ClientError


from ig_config import Config
from ig_client import IgClient
from ig_utils import IgPost, IgUtils, create_post_dataframe, save_post_dataframe, correct_orientation
from ig_post_manager import IgPostManager

In [ ]:
import pprint

In [2]:
# Configure logging (optional, but recommended)
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load project configuration file
config_file_path = r"C:\Users\manue\Documents\GitHubMLSN\sn-libraries\notebooks\ig_JK_config.json"  # Replace with the actual path
config = Config(config_file_path=config_file_path)  # Create Config object with path
username = config.get("username")
password = config.get("password")

# Authenticate using IgClient
igcl = IgClient(session_file=config.get("settings_file_path"))  
igcl.login(username, password)

#Instantiate objects
ig_utils = IgUtils(igcl)
post_manager = IgPostManager(igcl)

INFO:ig_client:Session loaded successfully.
INFO:ig_client:Connected Successfully!


In [3]:
# just one post as a test before automation

path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\IMG_3239.JPG"
correct_orientation(Path(path))

caption = "  #JKTravels"


In [4]:

location_comas = ig_utils.get_location_by_pk(250765185)


INFO:public_request:[None] [201] GET https://www.instagram.com/explore/locations/250765185/?__a=1&__d=dis
ERROR:public_request:Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/explore/locations/250765185/?__a=1&__d=dis) >>> 
INFO:public_request:[None] [201] GET https://www.instagram.com/explore/locations/250765185/?__a=1&__d=dis
ERROR:public_request:Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/explore/locations/250765185/?__a=1&__d=dis) >>> 
INFO:public_request:[None] [201] GET https://www.instagram.com/explore/locations/250765185/?__a=1&__d=dis
ERROR:public_request:Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/explore/locations/250765185/?__a=1&__d=dis) >>> 
INFO:instagrapi:https://i.instagram.com/api/v1/locations/250765185/location_info/
INFO:private_request:johnklanick [200] GET https://i.instagram.com/api/v1/locations/250765185/location_info/ (269.0.0.18.75, OnePlus 6T Dev)


In [ ]:
print(type(location_comas))

In [ ]:
print(location_comas.__dict__)

In [ ]:
# one photo upload_photo

photo_post = post_manager.upload_photo(photo_path=path, caption=caption, location=location_comas)
ig_photo_post = IgPost(photo_post)

In [ ]:

print(type(ig_photo_post))
pprint.pprint(ig_photo_post)

In [ ]:
#one video_post with upload_video
location_larcomar = ig_utils.get_location_by_pk(101440375504236)
path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\Belmond Miraflores Park.mp4"
caption = "Some tim ago I took this where today sits the wonderfull hotel @belmondmiraflorespark"
video_post = post_manager.upload_video(video_path=path, caption=caption, location=location_larcomar)

In [ ]:
ig_video_post = IgPost(video_post)
print(type(video_post))
pprint.pprint(video_post)

In [ ]:
# upload album post with upload_album
media_paths = [
    Path(r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\IMG_3211.JPG"),
    Path(r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\IMG_3227.JPG"),
    Path(r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\IMG_3197.JPG")  # Add more paths as needed
]
for path in media_paths:
    correct_orientation(path)  # Correct orientation before uploading
caption = """
street photography in Lima
"""
album_post = post_manager.upload_album(paths=media_paths, caption=caption, location=location_comas)


In [ ]:
ig_album_post = IgPost(album_post)
print(type(ig_album_post))
pprint.pprint(ig_album_post)

In [ ]:
# one reel upload_reel 

path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\dog.mp4"
caption = "dogs run on further than the Malecom"

reel = post_manager.upload_video(path, caption=caption, location= location_larcomar)

In [ ]:
ig_reel = IgPost(reel)
print(type(ig_reel))
print(ig_reel)

In [ ]:
# one reel with music upload_video_with_music

track_id: str = "318570269071669"

track = IgUtils.get_track_by_id(track_id)
print(type(track))
print(track)


In [ ]:
path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\dog.mp4"
caption = "dogs run on further than the Malecom"
reel_with_music = post_manager.upload_reel_with_music(path=path, caption=caption, track=track, location=location)

In [ ]:
ig_reel_with_music = IgPost(ig_reel_with_music)

In [ ]:
# --------------------------------------------------------------------
# --------------------------------------------------------------------
# 
# --------------------------------------------------------------------
# Read Excel into DataFrame


df_posts = pd.read_excel(r"C:\Users\manue\Documents\GitHubMLSN\sn-libraries\notebooks\JK_post_in_queue.xlsx")


In [ ]:
####   NOT RUN UNTIL LATER····

# Check if 'published' column exists in the DataFrame
if 'published' not in df_posts.columns:
    df_posts["published"] = False

# Load existing post history and combine with new data
df_posts = load_and_merge_post_history(df_posts)

print(df_posts['Mentions'].apply(type).unique())



In [ ]:
display(df_posts)

In [ ]:

# Replace NaN with empty string in the "Hashtags" and "Mentions" columns
df_posts['Mentions'] = df_posts['Mentions'].fillna('')
df_posts['Hashtags'] = df_posts['Hashtags'].fillna('')
print(df_posts['Mentions'].apply(type).unique())

In [ ]:
print(df_posts['Mentions'])

In [ ]:

import pprint
pprint.pprint(df_posts)


In [ ]:
# simplified for 1 post
published_posts = []
df_posts['failed_attempts'] = 0  # Initialize 'failed_attempts' column here

for _, row in df_posts[df_posts['published'] == False].head(2).iterrows():
    try:
        post = publish_post(post_manager, row)
        if post.published:  # Add to the list only if published successfully
            published_posts.append(post.to_dict())
        else:
            # If failed, update the DataFrame with failed attempts and timestamp
            df_posts.loc[df_posts['post_id'] == row['post_id'], 'failed_attempts'] = post.failed_attempts
            df_posts.loc[df_posts['post_id'] == row['post_id'], 'last_failed_attempt'] = post.last_failed_attempt

        time.sleep(random.randint(90, 900))
    except Exception as e:
        # Handle errors (log, retry, skip, or stop)
        logging.error(f"Error publishing post: {e}")
        row['published'] = False
        row['failed_attempts'] += 1
        row['last_failed_attempt'] = datetime.datetime.now()


In [ ]:


# Schedule posts
published_posts = []
df_posts['failed_attempts'] = 0  # Initialize 'failed_attempts' column here

for _, row in df_posts[df_posts['published'] == False].head(2).iterrows():
    try:
        post = publish_post(post_manager, row)
        if post.published:  # Add to the list only if published successfully
            published_posts.append(post.to_dict())
        else:
            # If failed, update the DataFrame with failed attempts and timestamp
            df_posts.loc[df_posts['post_id'] == row['post_id'], 'failed_attempts'] = post.failed_attempts
            df_posts.loc[df_posts['post_id'] == row['post_id'], 'last_failed_attempt'] = post.last_failed_attempt

        time.sleep(random.randint(90, 900))
    except Exception as e:
        # Handle errors (log, retry, skip, or stop)
        logging.error(f"Error publishing post: {e}")
        row['published'] = False
        row['failed_attempts'] += 1
        row['last_failed_attempt'] = datetime.datetime.now()


In [ ]:
print(post_manager)

In [ ]:

print("solo queda guardar el historico en algun sitio")
import pprint
pprint.pprint(published_posts)



In [ ]:



#Update Posts History
if os.path.exists(POSTS_HISTORY_FILE):
    with open(POSTS_HISTORY_FILE, "r", encoding="utf-8") as f:
        posts_data = json.load(f)
    posts_data["posts"].extend(published_posts)
    with open(POSTS_HISTORY_FILE, "w", encoding="utf-8") as f:
        json.dump(posts_data, f, ensure_ascii=False, indent=4)
else: 
    posts_data = {"posts": published_posts}
    with open(POSTS_HISTORY_FILE, "w", encoding="utf-8") as f:
        json.dump(posts_data, f, ensure_ascii=False, indent=4)

